In [1]:
import pandas as pd
from googletrans import Translator
import time

In [5]:
# 1. CSV 파일 읽기
df = pd.read_csv("data/전처리.csv", encoding="utf-8")

# 2. 데이터 탐색
print(df.info())
print(df.describe())

# 3. 숫자형 컬럼 변환
num_cols = ['1회 섭취량(g)', 'kcal', '탄수화물(g)', '단백질(g)', '지방(g)', '당류(g)']
df[num_cols] = df[num_cols].apply(pd.to_numeric, errors='coerce')

# 4. 결측치 확인 및 처리
print(df.isnull().sum())
df = df.dropna()

# 5. 중복 제거
df = df.drop_duplicates()

# 6. 숫자값 반올림 (예: 소수점 1자리)
df[num_cols] = df[num_cols].round(1)

# 7.고유 식별자 생성 – 예를 들어, 식품종류와 상세분류을 결합
df['고유ID'] = df['식품종류'] + "_" + df['상세분류']

# 8. 필요없는 columns 제거
df = df.drop(['Unnamed: 0', '식품종류', '상세분류'], axis=1)

# 9. column 정렬
df_col = ['고유ID', '식품명', '1회 섭취량(g)', 'kcal', '탄수화물(g)', '단백질(g)', '지방(g)', '당류(g)']
df = df[df_col]

# 10. 전처리 완료된 데이터를 새 CSV로 저장
df.to_csv("data/data1.csv", index=False, encoding="utf-8")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52940 entries, 0 to 52939
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  52940 non-null  int64  
 1   식품종류        52940 non-null  object 
 2   식품명         52940 non-null  object 
 3   상세분류        52940 non-null  object 
 4   1회 섭취량(g)   52940 non-null  float64
 5   kcal        52940 non-null  object 
 6   탄수화물(g)     52940 non-null  object 
 7   단백질(g)      52940 non-null  object 
 8   지방(g)       52940 non-null  object 
 9   당류(g)       52940 non-null  object 
dtypes: float64(1), int64(1), object(8)
memory usage: 4.0+ MB
None
         Unnamed: 0       1회 섭취량(g)
count  52940.000000    52940.000000
mean   26469.500000      378.158917
std    15282.605962    15055.322609
min        0.000000        0.030000
25%    13234.750000       70.000000
50%    26469.500000      100.000000
75%    39704.250000      189.000000
max    52939.000000  1000000.000000
Unnamed:

In [12]:
food_data = pd.read_csv('data/data1.csv')
food_data

,고유ID,식품명,1회 섭취량(g),kcal,탄수화물(g),단백질(g),지방(g),당류(g)
0,음식_육류구이,꿩불고기,500.0,368.8,39.7,33.5,8.5,16.9
1,음식_육류구이,닭갈비,400.0,595.6,44.9,45.9,25.8,21.2
2,음식_육류구이,닭갈비,300.0,558.5,23.1,45.5,31.6,8.5
3,음식_육류구이,닭꼬치,70.0,176.7,13.3,11.6,8.6,3.2
4,음식_채소류구이,더덕구이,100.0,184.0,31.1,3.1,5.2,11.6
...,...,...,...,...,...,...,...,...
42337,가공식품_캔디류,블루하와이 젤리,42.0,156.0,36.0,3.0,0.0,27.0
42338,가공식품_캔디류,라임모히또 젤리,70.0,264.0,61.0,5.0,0.0,44.0
42339,가공식품_캔디류,꼬마쮸 복숭아맛,40.0,175.0,36.0,0.0,3.6,29.0
42340,가공식품_캔디류,올리렉스 로젠지,120.0,300.0,120.0,0.0,0.0,0.0


# 병합

In [6]:
file_paths = {
    "diet": "data/체형(다이어트).csv",
    "maintain": "data/체형(근육량유지).csv",
    "muscle": "data/체형(근육).csv",
    "nutrition": "data/food_nutrition_data.csv",
    "extra": "data/data1.csv"
}

# 인코딩 방식 설정
encodings = {"diet": "cp949", "maintain": "cp949", "muscle": "cp949", "nutrition": "utf-8", "extra": "utf-8"}

# 체형별 식단 데이터 불러오기
diet_df = pd.read_csv(file_paths["diet"], encoding=encodings["diet"])
maintain_df = pd.read_csv(file_paths["maintain"], encoding=encodings["maintain"])
muscle_df = pd.read_csv(file_paths["muscle"], encoding=encodings["muscle"])

# 음식 영양소 데이터 불러오기
nutrition_df = pd.read_csv(file_paths["nutrition"], encoding=encodings["nutrition"])
extra_df = pd.read_csv(file_paths["extra"], encoding=encodings["extra"])

# 체형별 식단 데이터 통합
body_type_df = pd.concat([diet_df, maintain_df, muscle_df], ignore_index=True)

# 음식명 컬럼 정리
nutrition_df.rename(columns={"식품명(영문)": "식품명"}, inplace=True)
extra_df.rename(columns={"식품명": "식품명"}, inplace=True)

# 음식 영양 정보 병합
food_data = pd.merge(nutrition_df, extra_df, on="식품명", how="outer")

# 최종 데이터 저장
body_type_df.to_csv("data/merged_body_type.csv", index=False, encoding="utf-8")
food_data.to_csv("data/merged_food_nutrition.csv", index=False, encoding="utf-8")

print("✅ 데이터 병합 완료! \n체형별 식단: merged_body_type.csv\n음식 영양 정보: merged_food_nutrition.csv")

✅ 데이터 병합 완료! 
체형별 식단: merged_body_type.csv
음식 영양 정보: merged_food_nutrition.csv
